# Music Store Transactions



In [ ]:
%%capture
%load_ext sql

%sql sqlite:///chinook.db


This notebook contains SQL queries answering questions as part of DataQuest's project titled "Answering Business Questions with SQL". The database and schema used are pictured below and help answer the following questions:

1. Based on the current database, which are the top trending genres based on purchases?
2. Who are the best performing sales reps based on sales totals? Can you explain any discrepancy in the sales numbers and provide an explanation for why this variation is occurring?
3. Should the music store adopt a policy of selling specific tracks from albums as opposed to listing all tracks from an album? Compare the sales amounts for individual tracks vs. album purchases and advise whether the company should adopt the new policy.


![title](music_schema.png)




In [3]:
%%sql
select

name, 
type
from sqlite_master
where type in ("table","view")

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [31]:
%%sql

 with cte as(
    Select
            
    g.name as genre,
    i.billing_country as country, 
    cast(count(il.track_id)as float) as t_count

    from invoice i 

    inner join invoice_line il on i.invoice_id = il.invoice_id
    inner join track t on t.track_id = il.track_id
    inner join genre g on g.genre_id = t.genre_id

    where i.billing_country = "USA"
    group by g.name
 )
    
    
select 
*, 
round(100*cte.t_count/(Select sum(t_count) from cte),1) as t_pct

from cte

order by t_pct desc

limit 5


Done.


genre,country,t_count,t_pct
Rock,USA,561.0,53.4
Alternative & Punk,USA,130.0,12.4
Metal,USA,124.0,11.8
R&B/Soul,USA,53.0,5.0
Blues,USA,36.0,3.4


The top performing genres within the USA according to Chinook are 
Rock, Alternative&Punk, and Metal with percentages of sales of
53.4%, 12.4% and 11.8% respectively. These would be the recommended
genres to explore for the new album sales 

In [46]:

%%sql

select
    e.first_name || " "|| e.last_name as employee,
    e.title,
    round(sum(i.total),2) as total_sales, 
    cast(e.hire_date as date) as hire_year
from employee e 
    inner join customer c on e.employee_id = c.support_rep_id
    inner join invoice i on i.customer_id = c.customer_id

group by employee

Done.


employee,title,total_sales,hire_year
Jane Peacock,Sales Support Agent,1731.51,2017
Margaret Park,Sales Support Agent,1584.0,2017
Steve Johnson,Sales Support Agent,1393.92,2017


In [8]:
%%sql

with cte as (select
    e.employee_id,
    e.first_name || " "|| e.last_name as employee,
    e.title,
    round(sum(il.quantity*il.unit_price),2) as total_sales, 
    cast(e.hire_date as date) as hire_year, 
    max(i.invoice_date) as last_sale_date, 
    e.city ||", "|| e.country as Location
    
from employee e 
    inner join customer c on e.employee_id = c.support_rep_id
    inner join invoice i on i.customer_id = c.customer_id
    inner join invoice_line il on il.invoice_id = i.invoice_id
    inner join track t on t.track_id = il.track_id
    inner join genre g on g.genre_id = t.genre_id


group by employee
             
            )


select * from cte

Done.


employee_id,employee,title,total_sales,hire_year,last_sale_date,Location
3,Jane Peacock,Sales Support Agent,1731.51,2017,2020-12-30 00:00:00,"Calgary, Canada"
4,Margaret Park,Sales Support Agent,1584.0,2017,2020-12-29 00:00:00,"Calgary, Canada"
5,Steve Johnson,Sales Support Agent,1393.92,2017,2020-12-20 00:00:00,"Calgary, Canada"


These are the only sales reps in the company. Listed are their 
total sales and other relevant employee info 

In [24]:
%%sql

select
    e.employee_id,
    e.first_name || " "|| e.last_name as employee,
    g.name,
    round(sum(il.quantity*il.unit_price),2) as total_sales
    
from employee e 
    inner join customer c on e.employee_id = c.support_rep_id
    inner join invoice i on i.customer_id = c.customer_id
    inner join invoice_line il on il.invoice_id = i.invoice_id
    inner join track t on t.track_id = il.track_id
    inner join genre g on g.genre_id = t.genre_id
    
group by employee, g.name

limit 5


Done.


employee_id,employee,name,total_sales
3,Jane Peacock,Alternative,40.59
3,Jane Peacock,Alternative & Punk,152.46
3,Jane Peacock,Blues,28.71
3,Jane Peacock,Classical,18.81
3,Jane Peacock,Drama,0.99


Adding this view would be helpful to see which category each sales
rep is doing the best in. Adding this to the query above

In [11]:
%%sql
with genre_sub as (
    
    select
    e.employee_id,
    e.first_name || " "|| e.last_name as employee,
    g.name,
    round(sum(il.quantity*il.unit_price),2) as total_sales
    
from employee e 
    inner join customer c on e.employee_id = c.support_rep_id
    inner join invoice i on i.customer_id = c.customer_id
    inner join invoice_line il on il.invoice_id = i.invoice_id
    inner join track t on t.track_id = il.track_id
    inner join genre g on g.genre_id = t.genre_id
    
group by employee, g.name
), 

genre_max as(
    select 
    gs.employee_id, 
    gs.employee, 
    gs.name as genre,
    max(total_sales) as "highest selling genre"
    
    from genre_sub as gs

group by employee
)

select *
from genre_max; 

Done.


employee_id,employee,genre,highest selling genre
3,Jane Peacock,Rock,1035.54
4,Margaret Park,Rock,777.15
5,Steve Johnson,Rock,795.96


This combined query gives the highest selling genre per sales person

In [53]:
%%sql
with genre_sub as (
    
    select
    e.employee_id,
    e.first_name || " "|| e.last_name as employee,
    g.name,
    round(sum(il.quantity*il.unit_price),2) as total_sales
    
from employee e 
    inner join customer c on e.employee_id = c.support_rep_id
    inner join invoice i on i.customer_id = c.customer_id
    inner join invoice_line il on il.invoice_id = i.invoice_id
    inner join track t on t.track_id = il.track_id
    inner join genre g on g.genre_id = t.genre_id
    
group by employee, g.name
), 

genre_max as(
    select 
    gs.employee_id, 
    gs.employee, 
    gs.name as genre,
    max(total_sales) as highest_selling_genre
    
    from genre_sub as gs

group by employee
),

cte as (select
    e.employee_id,
    e.first_name || " "|| e.last_name as employee,
    e.title,
    round(sum(il.quantity*il.unit_price),2) as total_sales, 
    date(e.hire_date) as hire_year, 
    max(i.invoice_date) as last_sale_date, 
    e.city ||", "|| e.country as Location
    
from employee e 
    inner join customer c on e.employee_id = c.support_rep_id
    inner join invoice i on i.customer_id = c.customer_id
    inner join invoice_line il on il.invoice_id = i.invoice_id
    inner join track t on t.track_id = il.track_id
    inner join genre g on g.genre_id = t.genre_id


group by employee
             
            )

select 

    cte.*,
    round(((select max(cte.total_sales) from cte) - cte.total_sales)/
          (select max(cte.total_sales) from cte),2)
    as delta_to_max,
    gm.highest_selling_genre as highest_genre_sales,
    gm.genre

from cte left join genre_max gm on cte.employee_id = gm.employee_id

Done.


employee_id,employee,title,total_sales,hire_year,last_sale_date,Location,delta_to_max,highest_genre_sales,genre
3,Jane Peacock,Sales Support Agent,1731.51,2017-04-01,2020-12-30 00:00:00,"Calgary, Canada",0.0,1035.54,Rock
4,Margaret Park,Sales Support Agent,1584.0,2017-05-03,2020-12-29 00:00:00,"Calgary, Canada",0.09,777.15,Rock
5,Steve Johnson,Sales Support Agent,1393.92,2017-10-17,2020-12-20 00:00:00,"Calgary, Canada",0.19,795.96,Rock


The bottom 2 sales rep differ by 10% and 20% from the top performer. However, after controlling for location of sales, years of experience, music genre sold, it seems that the performances between the reps are not significantly different. 

The reps all started in the same year, all sell in the same city in
Canada, and all have Rock as their highest performing genre. 

The two differences that could be attributed to the performance, and would lead to the assertion of their performance being the same, is that 

1. the top 2 sales rep have their last sales date 7+ days later than the worst performer. This rep may have been on vacation and this would lead to seemingly worse sales performance year to date if unaccounted for.

2. the start date of the worst performer almost 7 months later than the top performer. Assuming they have the same weekly rate, this would also cause the discrepancy in sales data. 

Next steps to confirm this would be to determine the average weekly performance for the past month(at least) and confirm whether or not this extra week would have caught the worst performer up to the top 

In [47]:
%%sql
select
    e.employee_id,
    e.first_name || " "|| e.last_name as employee,
    e.title,
    round(sum(il.quantity*il.unit_price)/104,2) as weekly_sales 
    
    
from employee e 
    inner join customer c on e.employee_id = c.support_rep_id
    inner join invoice i on i.customer_id = c.customer_id
    inner join invoice_line il on il.invoice_id = i.invoice_id
    inner join track t on t.track_id = il.track_id
    inner join genre g on g.genre_id = t.genre_id
    
where i.invoice_date between '2018-01-01'and '2019-12-30'
group by employee

Done.


employee_id,employee,title,weekly_sales
3,Jane Peacock,Sales Support Agent,7.62
4,Margaret Park,Sales Support Agent,7.6
5,Steve Johnson,Sales Support Agent,7.41


In [55]:
%%sql
Select 
7.6*4 as Missing_Potential_Sales_4, 
1584+7.6*4 as Potential_Total_Sales_4, 
(1731 - (1584+7.6*4))/1731 as delta_to_top_4,

7.4*28 as Missing_Potential_Sales_5, 
1393+7.4*28 as Potential_Total_Sales_5, 
(1731 - (1393+7.4*28))/1731 as new_delta_to_top_5

Done.


Missing_Potential_Sales_4,Potential_Total_Sales_4,delta_to_top_4,Missing_Potential_Sales_5,Potential_Total_Sales_5,new_delta_to_top_5
30.4,1614.4,0.06735990756787978,207.20000000000002,1600.2,0.07556325823223567


After computing the weekly sales across two full years in which all sales reps were working, we see that had the worst performers(to date) started at the same, they would be within 8% of the top performer

In [83]:
%%sql
            
with 
cte as(
    select
        case 
             when count(distinct c.customer_id) < 2 then "Other"
             else c.country 
             end as country,
        count(distinct c.customer_id) as total_customers,
        round(sum(i.total),2) as total_sales,
        round(sum(i.total),2)/count(distinct c.customer_id) as avg_sales_cus,
        round(avg(i.total),2) as avg_order_value

    from customer c 
    inner join invoice i on c.customer_id = i.customer_id

    group by c.country
           ),
cte2 as (
    select
        cte.country,
        sum(cte.total_customers) as total_customers,
        avg(cte.total_sales) as total_sales,
        avg(cte.avg_sales_cus) as avg_sales,
        avg(cte.avg_order_value) as avg_order_value
    from cte
    group by cte.country

)

select*from cte2 order by cte2.avg_sales



Done.


country,total_customers,total_sales,avg_sales,avg_order_value
Canada,8,535.59,66.94875,7.05
Other,15,72.99600000000001,72.99600000000001,7.444
France,5,389.07,77.814,7.78
USA,13,1040.49,80.03769230769231,7.94
United Kingdom,3,245.52,81.84,8.77
Germany,4,334.62,83.655,8.16
Brazil,5,427.68,85.536,7.01
India,2,183.15,91.575,8.72
Portugal,2,185.13,92.565,6.38
Czech Republic,2,273.24,136.62,9.11


There is a lack of customer base for Czech Republic, Portugal, and India, so there are opportunities for marketing or expansion operations in those countries.  

In [87]:
%%sql
            
with 
cte as(
    select
        case 
             when count(distinct c.customer_id) < 2 then "Other"
             else c.country 
             end as country,
        count(distinct c.customer_id) as total_customers,
        round(sum(i.total),2) as total_sales,
        round(sum(i.total),2)/count(distinct c.customer_id) as avg_sales_cus,
        round(avg(i.total),2) as avg_order_value

    from customer c 
    inner join invoice i on c.customer_id = i.customer_id

    group by c.country
           ),
cte2 as (
    select
        cte.country,
        sum(cte.total_customers) as total_customers,
        avg(cte.total_sales) as total_sales,
        avg(cte.avg_sales_cus) as avg_sales_per_cus,
        avg(cte.avg_order_value) as avg_order_value
    from cte
    group by cte.country

)

select*
from cte2 
where cte2.total_customers >4 and country != "Other"
order by cte2.avg_sales_per_cus



Done.


country,total_customers,total_sales,avg_sales_per_cus,avg_order_value
Canada,8,535.59,66.94875,7.05
France,5,389.07,77.814,7.78
USA,13,1040.49,80.03769230769231,7.94
Brazil,5,427.68,85.536,7.01


In addition to cultivating burgeoning markeys, we should keep in mind improvements for our current, steady markets defined as those with customer greater than 4. The US is doing exeptionally well with 13 customers and avg sales of 80 whereas Canada and France seem to be lagging (if we consider their total customers to be of the same order of magnitude as USA)

In [30]:
%%sql

select

i.invoice_id, 
i.total,
il.invoice_line_id, 
il.track_id, 
a.album_id, 
count(distinct il.track_id) as track_count_iv

from invoice i
inner join invoice_line il on il.invoice_id = i.invoice_id
inner join track t on t.track_id = il.track_id
inner join album a on a.album_id = t.album_id

group by i.invoice_id

limit 10

Done.


invoice_id,total,invoice_line_id,track_id,album_id,track_count_iv
1,15.84,16,1173,91,16
2,9.9,26,2651,215,10
3,1.98,28,2646,214,2
4,7.92,36,2514,203,8
5,16.83,53,2002,163,17
6,1.98,55,30,5,2
7,10.89,66,42,6,11
8,9.9,76,1560,125,10
9,8.91,85,196,20,9
10,1.98,87,3290,257,2


Seeing how the invoice tables join together

In [23]:
%%sql

select
a.album_id, 
t.track_id

from album a
left join track t on a.album_id = t.album_id

limit 10



Done.


album_id,track_id
1,1
1,6
1,7
1,8
1,9
1,10
1,11
1,12
1,13
1,14


Seeing how the albums table joins with the tracks table

In [26]:
%%sql

select
a.album_id,
count(t.track_id)

from album a
left join track t on a.album_id = t.album_id

group by a.album_id

limit 10


Done.


album_id,count(t.track_id)
1,10
2,1
3,3
4,8
5,15
6,13
7,12
8,14
9,8
10,14


Counting the number of tracks that exist for each album

In [61]:
%%sql

with 
invoice_table as(
    select
        i.invoice_id, 
        i.total,
        il.invoice_line_id, 
        il.track_id, 
        a.album_id, 
        count(distinct il.track_id) as track_count_iv
    from invoice i
        inner join invoice_line il on il.invoice_id = i.invoice_id
        inner join track t on t.track_id = il.track_id
        inner join album a on a.album_id = t.album_id
group by i.invoice_id
),

albums_table as(
    select
        a.album_id,
        count(t.track_id) as track_count_at

    from album a
        left join track t on a.album_id = t.album_id

group by a.album_id
)

select 
    *

from(
    select 
        it.*, 
        at.track_count_at, 
        case 
            when it.track_count_iv = at.track_count_at then 1
            else 0
        end as whole_album

    from invoice_table it
    inner join albums_table at on at.album_id = it.album_id
) as sub

limit 15

Done.


invoice_id,total,invoice_line_id,track_id,album_id,track_count_iv,track_count_at,whole_album
1,15.84,16,1173,91,16,16,1
2,9.9,26,2651,215,10,14,0
3,1.98,28,2646,214,2,11,0
4,7.92,36,2514,203,8,17,0
5,16.83,53,2002,163,17,17,1
6,1.98,55,30,5,2,15,0
7,10.89,66,42,6,11,13,0
8,9.9,76,1560,125,10,16,0
9,8.91,85,196,20,9,11,0
10,1.98,87,3290,257,2,12,0


This is the subquery table that we will be counting from whether or not there were whole album purchases

In [69]:
%%sql

with 
invoice_table as(
    select
        i.invoice_id, 
        i.total,
        il.invoice_line_id, 
        il.track_id, 
        a.album_id, 
        count(distinct il.track_id) as track_count_iv
    from invoice i
        inner join invoice_line il on il.invoice_id = i.invoice_id
        inner join track t on t.track_id = il.track_id
        inner join album a on a.album_id = t.album_id
group by i.invoice_id
),

albums_table as(
    select
        a.album_id,
        count(t.track_id) as track_count_at

    from album a
        left join track t on a.album_id = t.album_id

group by a.album_id
)

select 
    count(sub.whole_album) as invoices,
    sum(sub.whole_album) as album_purchases,
    round(cast(sum(sub.whole_album) as float)/count(sub.whole_album),3) as Whole_Albums, 
    round(1-cast(sum(sub.whole_album) as float)/count(sub.whole_album),3) as Singles 

from(
    select 
        it.*, 
        at.track_count_at, 
        case 
            when it.track_count_iv = at.track_count_at then 1
            else 0
        end as whole_album

    from invoice_table it
    inner join albums_table at on at.album_id = it.album_id
) as sub


Done.


invoices,album_purchases,Whole_Albums,Singles
614,123,0.2,0.8


In [70]:
%%sql

with 
invoice_table as(
    select
        i.invoice_id, 
        i.total,
        il.invoice_line_id, 
        il.track_id, 
        a.album_id, 
        count(distinct il.track_id) as track_count_iv
    from invoice i
        inner join invoice_line il on il.invoice_id = i.invoice_id
        inner join track t on t.track_id = il.track_id
        inner join album a on a.album_id = t.album_id
group by i.invoice_id, a.album_id
),

albums_table as(
    select
        a.album_id,
        count(t.track_id) as track_count_at

    from album a
        left join track t on a.album_id = t.album_id

group by a.album_id
)

select *
from(
    select 
        it.*, 
        at.track_count_at, 
        case 
            when it.track_count_iv = at.track_count_at then 1
            else 0
        end as whole_album

    from invoice_table it
    inner join albums_table at on at.album_id = it.album_id
) as sub

limit 25


Done.


invoice_id,total,invoice_line_id,track_id,album_id,track_count_iv,track_count_at,whole_album
1,15.84,16,1173,91,16,16,1
2,9.9,23,201,20,1,11,0
2,9.9,25,392,34,1,17,0
2,9.9,18,482,39,1,21,0
2,9.9,24,819,66,1,10,0
2,9.9,21,1119,73,1,30,0
2,9.9,20,1641,134,1,10,0
2,9.9,22,2324,190,1,16,0
2,9.9,26,2651,215,1,14,0
2,9.9,19,2701,218,1,15,0


This new table now groups by invoice ID and then by album id. This lists all of the combinations of invoices with albums and accounts for those purchases of albums that are only one single in length. As you can see, we now also count single song albums that are part of a larger set of song purchases within one invoice. This should increase the number of album purchases

In [68]:
%%sql

with 
invoice_table as(
    select
        i.invoice_id, 
        i.total,
        il.invoice_line_id, 
        il.track_id, 
        a.album_id, 
        count(distinct il.track_id) as track_count_iv
    from invoice i
        inner join invoice_line il on il.invoice_id = i.invoice_id
        inner join track t on t.track_id = il.track_id
        inner join album a on a.album_id = t.album_id
group by i.invoice_id, a.album_id
),

albums_table as(
    select
        a.album_id,
        count(t.track_id) as track_count_at

    from album a
        left join track t on a.album_id = t.album_id

group by a.album_id
)

select 
    count(distinct sub.invoice_id) as invoices, 
    sum(sub.whole_album) as album_purchases,
    round(cast(sum(sub.whole_album) as float)/count(distinct sub.invoice_id),3) as Whole_Albums, 
    round(1-cast(sum(sub.whole_album) as float)/count(distinct sub.invoice_id),3) as Singles 
from(
    select 
        it.*, 
        at.track_count_at, 
        case 
            when it.track_count_iv = at.track_count_at then 1
            else 0
        end as whole_album

    from invoice_table it
    inner join albums_table at on at.album_id = it.album_id
) as sub


Done.


invoices,album_purchases,Whole_Albums,Singles
614,208,0.339,0.661


After accounting for those 1 song albums, it seems that the number of whole album purchases almost doubled. However despite this drastic change, it may not change the recommendation above for two reasons:

1. The single song purchases still claim a majority of the invoices
2. The purchase of a single song album may not be considered the same "purchase" as buying a multisong album due to the length differences of the album. It does not tell the music shop whether or not customers prefer buying artist's songs in bulk or piecemeal which is the root question, instead it actually confounds it. Customers may prefer buying songs in bulk and would have bought this album regardless of length or customers may prefer buying piecemeal and got lucky with a short album

In [59]:
%%sql

with 
invoice_table as(
    select
        i.invoice_id, 
        i.total,
        il.invoice_line_id, 
        il.track_id, 
        a.album_id, 
        count(distinct il.track_id) as track_count_iv
    from invoice i
        inner join invoice_line il on il.invoice_id = i.invoice_id
        inner join track t on t.track_id = il.track_id
        inner join album a on a.album_id = t.album_id
group by i.invoice_id, a.album_id
),

albums_table as(
    select
        a.album_id,
        count(t.track_id) as track_count_at

    from album a
        left join track t on a.album_id = t.album_id

group by a.album_id
)

select 
    sum(sub.whole_album),
    count(distinct sub.whole_album) as Whole_Albums
from(
    select 
        it.*, 
        at.track_count_at, 
        case 
            when it.track_count_iv = at.track_count_at then 1
            else 0
        end as whole_album

    from invoice_table it
    inner join albums_table at on at.album_id = it.album_id
) as sub


Done.


sum(sub.whole_album),Whole_Albums
208,2
